In [1]:
%load_ext watermark
%watermark

2019-05-24T10:36:28+01:00

CPython 3.6.8
IPython 7.5.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-50-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


# Aprendizaje online con scikit-learn

Hemos visto como podemos hacer cálculos sencillos con datasets medianos usando pandas. De la misma manera podemos entrenar (ciertos) modelos predictivos de forma iterativa usando scikit-learn.

Ésto requiere el uso de modelos cuyos algoritmos permiten el [aprendizaje incremental](https://scikit-learn.org/0.15/modules/scaling_strategies.html), esto es, el poder entrenar el módelo con pequeños incrementos de datos a la vez en vez de tener acceso a todo el dataset para entrenar.

En scikit-learn, esto significa que podemos usar aquellos modelos que tienen la función `partial_fit` de forma iterativa.

In [2]:
import pandas as pd

In [3]:
path_dataset = "../data/nyc_taxi_data_2014.csv"

pd.read_csv(path_dataset, nrows=1000).head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0.0,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0.0,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0.0,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0.0,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0.0,8.75


Supongamos que queremos hacer un modelo predictivo que predice la cantidad de de propina en función de la empresa procesadora (`vendor_id`), el tipo de pago (`payment_type`), el número de pasajeros (`passenger_count`) y la distancia (`trip_distance`)

In [4]:
bloque  = pd.read_csv(path_dataset, nrows=1000, 
                      header=None, 
                      skiprows=1,
                      usecols =[0, 3, 4, 15], 
                      names=["vendor_id", "passenger_count", "trip_distance", "tip_amount"])
bloque.head()

,vendor_id,passenger_count,trip_distance,tip_amount
0,CMT,1,0.7,1.40
1,CMT,1,1.4,1.90
2,CMT,2,2.3,1.50
3,CMT,1,1.7,1.70
4,CMT,1,0.9,1.75


In [5]:
bloque.dtypes

vendor_id           object
passenger_count      int64
trip_distance      float64
tip_amount         float64
dtype: object

Igual que hemos visto antes, vamos a hacer una funcion que lea un bloque de datos

In [6]:
def leer_bloque(filas_bloque, skiprows):
    return pd.read_csv(path_dataset, 
                          nrows=filas_bloque, 
                          header=None, 
                          skiprows=skiprows+1, # +1 por el header
                          usecols =[0, 3, 4, 15], 
                          names=["vendor_id", "passenger_count", 
                                 "trip_distance", "tip_amount"]                      
                      )

bloque = leer_bloque(filas_bloque=1000, skiprows=0)
bloque.head()

,vendor_id,passenger_count,trip_distance,tip_amount
0,CMT,1,0.7,1.40
1,CMT,1,1.4,1.90
2,CMT,2,2.3,1.50
3,CMT,1,1.7,1.70
4,CMT,1,0.9,1.75


Vemos que tenemos que convertir la variable categorica `vendor_id`. Para esto podemos usar la codificacion OneHot.

In [7]:
variable_objetivo = "tip_amount"
variables_independientes = ["vendor_id", "passenger_count", "trip_distance"]

In [8]:
X = bloque[variables_independientes]
y = bloque[variable_objetivo]

In [9]:
from sklearn.preprocessing import OneHotEncoder 

In [10]:
encoder = OneHotEncoder(sparse=False)

In [11]:
encoder.fit_transform(bloque[["vendor_id"]])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

Vemos que el codificador one hot ha convertido la variable en un array con solo un elemento `[1.]`, por que?

Muy sencillo, sólo hemos leido las 1000 primeras filas del dataset

In [12]:
bloque.shape

(1000, 4)

Y las primeras filas solo tienen un valor de la variable.

In [13]:
bloque.vendor_id.value_counts()

CMT    1000
Name: vendor_id, dtype: int64

Y si leemos ahora las siguientes 1000 filas?

In [14]:
bloque = leer_bloque(filas_bloque=1000, skiprows=1000)
bloque.vendor_id.value_counts()

CMT    877
VTS    123
Name: vendor_id, dtype: int64

Vemos que la variable ahora tiene dos valores (niveles), "CMT" y "VTS", con lo cual si queremos leer este bloque y codificarlo con el mismo encoder, nos dará un error:

In [15]:
encoder.transform(bloque[["vendor_id"]])

ValueError: Found unknown categories ['VTS'] in column 0 during transform

¿Como podemos evitar esto? 

Una manera de evitar ésto es conociendo los posibles valores de antemano, para esto podemos tomar una muestra representativa del dataset, encontrar los valores unicos y usar esos (y ninguno más).

In [16]:
# tomamos un dataset mas grande y vemos los distintos niveles de la variable vendor_id
pd.read_csv("../data/nyc_taxi_data_2014.csv", nrows=1000000, header=None, skiprows=1,
                      usecols =[0], names=["vendor_id"]).vendor_id.value_counts()

CMT    999862
VTS       138
Name: vendor_id, dtype: int64

Ahora creamos el encoder pero especificando los valores aceptables, y especificando que devuelva NaN cuando aparezca un valor desconocido:

In [17]:
codificador = OneHotEncoder(sparse=False, categories=[["CMT", "VTS"]], 
                            handle_unknown="ignore")

Ahora podemos codificar sin problemas

In [18]:
codificador.fit(bloque[["vendor_id"]]).transform([["CMT"], ["VTS"],["NUEVO_VALOR"]])

array([[1., 0.],
       [0., 1.],
       [0., 0.]])

Ahora vamos a crear un pipeline de scikit-learn capaz de funcionar de manera incremental, esto es, leyendo bloques de datos iterativamente.

In [20]:
from sklearn.pipeline import make_pipeline, make_union
from mlxtend.feature_selection import ColumnSelector
from sklearn.preprocessing import StandardScaler


codificador_pipeline =  make_pipeline(
    make_union(
        make_pipeline(
            ColumnSelector(cols=["vendor_id"]),
            OneHotEncoder(sparse=False, categories=[["CMT", "VTS"]], handle_unknown="ignore")
        ),
        make_pipeline(
            ColumnSelector(cols=["passenger_count", "trip_distance"]),
            StandardScaler()
        )  
    )
)

In [21]:
codificador_pipeline.fit_transform(X)

array([[ 1.        ,  0.        , -0.40423263, -0.73872659],
       [ 1.        ,  0.        , -0.40423263, -0.50511077],
       [ 1.        ,  0.        ,  1.5298948 , -0.20474758],
       ...,
       [ 1.        ,  0.        , -0.40423263,  1.03007889],
       [ 1.        ,  0.        ,  1.5298948 ,  0.09561562],
       [ 1.        ,  0.        , -0.40423263, -0.60523184]])

Ya tenemos un pipeline para codificar en bloque, ahora nos falta un estimador. Podemos elegir cualquier estimador de scikit-learn que soporte `partial_fit()`, por ejemplo, el [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html), que es un metodo de regresion lineal mediante descenso de gradiente.

In [22]:
from sklearn.linear_model import SGDRegressor

estimador = SGDRegressor()

In [24]:
estimador.partial_fit

<bound method BaseSGDRegressor.partial_fit of SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)>

In [25]:
estimador.intercept_, estimador.coef_

AttributeError: 'SGDRegressor' object has no attribute 'intercept_'

In [26]:
def entrenamiento_bloque(bloque, codificador, estimador):
    y = bloque[variable_objetivo]
    X_codificado = codificador.transform(bloque[variables_independientes])
    estimador.partial_fit(X_codificado, y)

Ahora podemos entrenar el estimador con un bloque de datos:

In [27]:
entrenamiento_bloque(bloque, codificador_pipeline, estimador)

In [28]:
estimador.intercept_, estimador.coef_

(array([1.25940545]), array([1.14620074, 0.1129706 , 0.04048952, 1.31331589]))

Ahora tenemos que usar un método similar al que hemos visto con pandas incremental, pero entrenando el estimador para cada bloque. De esta forma, conseguimos entrenar un modelo predictivo con un dataset grande pero sin tener que tener un ordenador muy potente.

In [29]:
%%time

filas_bloque = 2000000 # reducir esto si os quedais sin memoria
n_bloque = 0
n_total_filas = 0
estimador = SGDRegressor()

while 1:
    bloque = leer_bloque(filas_bloque=filas_bloque, skiprows=filas_bloque * n_bloque)
    if bloque.empty:
        break
    else:
        entrenamiento_bloque(bloque, codificador_pipeline, estimador)
    n_bloque += 1
    print(n_bloque * filas_bloque)

2000000
4000000
6000000
8000000
10000000
12000000
14000000
16000000
CPU times: user 1min 34s, sys: 4.06 s, total: 1min 38s
Wall time: 1min 42s


Ahora podemos predecir de forma igual a con cualquier modelo de sklearn

In [30]:
bloque_pred = pd.read_csv(path_dataset,
                         nrows=100, 
                         skiprows=1, 
                         header=None, 
                         usecols =[0, 3, 4], 
                         names=["vendor_id", "passenger_count", "trip_distance"])

In [31]:
X_transformado = codificador_pipeline.transform(bloque_pred)
estimador.predict(X_transformado)

array([1.16844723, 1.43736915, 1.75469595, 1.55262141, 1.24528206,
       1.24528206, 2.28255236, 1.70629108, 2.20571752, 1.78312592,
       4.54918003, 2.1673001 , 1.56260886, 1.40893919, 1.84151824,
       5.20227614, 1.55262141, 2.70514396, 1.51420399, 7.77624317,
       0.8995253 , 1.70629108, 6.39321611, 1.51420399, 1.39895174,
       1.39895174, 1.15000471, 1.13002981, 2.35938719, 1.59103883,
       1.36053432, 1.78312592, 1.10159984, 2.55147428, 1.89837817,
       1.89837817, 4.08817101, 1.62945625, 7.31523415, 3.8576665 ,
       2.03360533, 1.43736915, 1.24528206, 1.55262141, 3.58874458,
       1.47578657, 1.09161239, 1.13002981, 2.82039621, 4.24184069,
       1.47578657, 1.56260886, 4.62601487, 1.89837817, 1.39895174,
       2.47463945, 1.78312592, 1.85996075, 6.04745935, 2.24413494,
       3.05090072, 1.39895174, 2.47463945, 2.12888268, 1.2168521 ,
       1.79311337, 2.51305687, 0.8995253 , 1.09161239, 1.40893919,
       5.01018905, 2.35938719, 2.93564847, 1.98520046, 2.93564